### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

C:\Users\rfamo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [10]:
# Look at AFFILIATION value counts for binning
affiliate_count = application_df.AFFILIATION.value_counts()
affiliate_count

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [11]:
# Determine which values to replace if counts are less than ..?
replace_affiliate = list(affiliate_count[affiliate_count < 100].index)

# Replace in dataframe
for cls in replace_affiliate:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [13]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               3
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [14]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\rfamo\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\rfamo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [16]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

C:\Users\rfamo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 125
hidden_nodes_layer2 = 50

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               4750      
                                                                 
 dense_1 (Dense)             (None, 50)                6300      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75)

Epoch 1/75
804/804 [==============================] - 3s 2ms/step - loss: 0.5672 - accuracy: 0.7222
Epoch 2/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5557 - accuracy: 0.7299
Epoch 3/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7313
Epoch 4/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7334
Epoch 5/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7332
Epoch 6/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7335
Epoch 7/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7344
Epoch 8/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7344
Epoch 9/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7346
Epoch 10/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7349

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5684 - accuracy: 0.7236 - 581ms/epoch - 2ms/step
Loss: 0.5684120059013367, Accuracy: 0.7236151695251465


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Original/',exist_ok=True)
checkpoint_path = 'checkpoints/Original/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 90/804 [==>...........................] - ETA: 1s - loss: 0.5248 - accuracy: 0.7476
Epoch 1: saving model to checkpoints/Original\weights.01.hdf5
178/804 [=====>........................] - ETA: 1s - loss: 0.5300 - accuracy: 0.7410
Epoch 1: saving model to checkpoints/Original\weights.01.hdf5
294/804 [=========>....................] - ETA: 0s - loss: 0.5286 - accuracy: 0.7445
Epoch 1: saving model to checkpoints/Original\weights.01.hdf5
399/804 [=============>................] - ETA: 0s - loss: 0.5308 - accuracy: 0.7415
Epoch 1: saving model to checkpoints/Original\weights.01.hdf5
480/804 [================>.............] - ETA: 0s - loss: 0.5328 - accuracy: 0.7418
Epoch 1: saving model to checkpoints/Original\weights.01.hdf5
590/804 [=====================>........] - ETA: 0s - loss: 0.5331 - accuracy: 0.7423
Epoch 1: saving model to checkpoints/Original\weights.01.hdf5
689/804 [========================>.....] - ETA: 0s - loss: 0.5349 - accuracy: 0.7408
Epoch 1: saving model

649/804 [=======================>......] - ETA: 0s - loss: 0.5353 - accuracy: 0.7396
Epoch 13: saving model to checkpoints/Original\weights.13.hdf5
731/804 [==========================>...] - ETA: 0s - loss: 0.5356 - accuracy: 0.7387
Epoch 13: saving model to checkpoints/Original\weights.13.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5330 - accuracy: 0.7407
Epoch 14/100
 27/804 [>.............................] - ETA: 1s - loss: 0.5355 - accuracy: 0.7535
Epoch 14: saving model to checkpoints/Original\weights.14.hdf5
119/804 [===>..........................] - ETA: 1s - loss: 0.5290 - accuracy: 0.7526
Epoch 14: saving model to checkpoints/Original\weights.14.hdf5
216/804 [=======>......................] - ETA: 0s - loss: 0.5350 - accuracy: 0.7446
Epoch 14: saving model to checkpoints/Original\weights.14.hdf5
346/804 [===========>..................] - ETA: 0s - loss: 0.5322 - accuracy: 0.7449
Epoch 14: saving model to checkpoints/Original\weights.14.hdf5
413/804 [=

291/804 [=========>....................] - ETA: 0s - loss: 0.5373 - accuracy: 0.7383
Epoch 26: saving model to checkpoints/Original\weights.26.hdf5
395/804 [=============>................] - ETA: 0s - loss: 0.5348 - accuracy: 0.7394
Epoch 26: saving model to checkpoints/Original\weights.26.hdf5
480/804 [================>.............] - ETA: 0s - loss: 0.5335 - accuracy: 0.7401
Epoch 26: saving model to checkpoints/Original\weights.26.hdf5
581/804 [====================>.........] - ETA: 0s - loss: 0.5341 - accuracy: 0.7388
Epoch 26: saving model to checkpoints/Original\weights.26.hdf5
685/804 [========================>.....] - ETA: 0s - loss: 0.5324 - accuracy: 0.7408
Epoch 26: saving model to checkpoints/Original\weights.26.hdf5
798/804 [============================>.] - ETA: 0s - loss: 0.5329 - accuracy: 0.7407
Epoch 26: saving model to checkpoints/Original\weights.26.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7409
Epoch 27/100
 92/804 [=

804/804 [==============================] - 1s 2ms/step - loss: 0.5324 - accuracy: 0.7418
Epoch 39/100
 31/804 [>.............................] - ETA: 1s - loss: 0.5376 - accuracy: 0.7349
Epoch 39: saving model to checkpoints/Original\weights.39.hdf5
120/804 [===>..........................] - ETA: 1s - loss: 0.5321 - accuracy: 0.7375
Epoch 39: saving model to checkpoints/Original\weights.39.hdf5
235/804 [=======>......................] - ETA: 1s - loss: 0.5330 - accuracy: 0.7403
Epoch 39: saving model to checkpoints/Original\weights.39.hdf5
341/804 [===========>..................] - ETA: 0s - loss: 0.5323 - accuracy: 0.7388
Epoch 39: saving model to checkpoints/Original\weights.39.hdf5
420/804 [==============>...............] - ETA: 0s - loss: 0.5314 - accuracy: 0.7406
Epoch 39: saving model to checkpoints/Original\weights.39.hdf5
547/804 [===================>..........] - ETA: 0s - loss: 0.5292 - accuracy: 0.7435
Epoch 39: saving model to checkpoints/Original\weights.39.hdf5
632/804 [=

483/804 [=================>............] - ETA: 0s - loss: 0.5332 - accuracy: 0.7423
Epoch 51: saving model to checkpoints/Original\weights.51.hdf5
573/804 [====================>.........] - ETA: 0s - loss: 0.5358 - accuracy: 0.7399
Epoch 51: saving model to checkpoints/Original\weights.51.hdf5
695/804 [========================>.....] - ETA: 0s - loss: 0.5340 - accuracy: 0.7415
Epoch 51: saving model to checkpoints/Original\weights.51.hdf5
785/804 [============================>.] - ETA: 0s - loss: 0.5319 - accuracy: 0.7428
Epoch 51: saving model to checkpoints/Original\weights.51.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7424
Epoch 52/100
 86/804 [==>...........................] - ETA: 1s - loss: 0.5283 - accuracy: 0.7413
Epoch 52: saving model to checkpoints/Original\weights.52.hdf5
176/804 [=====>........................] - ETA: 1s - loss: 0.5356 - accuracy: 0.7370
Epoch 52: saving model to checkpoints/Original\weights.52.hdf5
286/804 [=

137/804 [====>.........................] - ETA: 1s - loss: 0.5364 - accuracy: 0.7352
Epoch 64: saving model to checkpoints/Original\weights.64.hdf5
243/804 [========>.....................] - ETA: 1s - loss: 0.5233 - accuracy: 0.7469
Epoch 64: saving model to checkpoints/Original\weights.64.hdf5
327/804 [===========>..................] - ETA: 0s - loss: 0.5241 - accuracy: 0.7472
Epoch 64: saving model to checkpoints/Original\weights.64.hdf5
435/804 [===============>..............] - ETA: 0s - loss: 0.5280 - accuracy: 0.7440
Epoch 64: saving model to checkpoints/Original\weights.64.hdf5
534/804 [==================>...........] - ETA: 0s - loss: 0.5298 - accuracy: 0.7433
Epoch 64: saving model to checkpoints/Original\weights.64.hdf5
641/804 [======================>.......] - ETA: 0s - loss: 0.5319 - accuracy: 0.7413
Epoch 64: saving model to checkpoints/Original\weights.64.hdf5
720/804 [=========================>....] - ETA: 0s - loss: 0.5313 - accuracy: 0.7417
Epoch 64: saving model to c

681/804 [========================>.....] - ETA: 0s - loss: 0.5348 - accuracy: 0.7398
Epoch 76: saving model to checkpoints/Original\weights.76.hdf5
784/804 [============================>.] - ETA: 0s - loss: 0.5329 - accuracy: 0.7414
Epoch 76: saving model to checkpoints/Original\weights.76.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7421
Epoch 77/100
 90/804 [==>...........................] - ETA: 1s - loss: 0.5324 - accuracy: 0.7417
Epoch 77: saving model to checkpoints/Original\weights.77.hdf5
195/804 [======>.......................] - ETA: 1s - loss: 0.5276 - accuracy: 0.7447
Epoch 77: saving model to checkpoints/Original\weights.77.hdf5
272/804 [=========>....................] - ETA: 1s - loss: 0.5308 - accuracy: 0.7438
Epoch 77: saving model to checkpoints/Original\weights.77.hdf5
380/804 [=============>................] - ETA: 0s - loss: 0.5287 - accuracy: 0.7437
Epoch 77: saving model to checkpoints/Original\weights.77.hdf5
494/804 [=

337/804 [===========>..................] - ETA: 0s - loss: 0.5338 - accuracy: 0.7439
Epoch 89: saving model to checkpoints/Original\weights.89.hdf5
443/804 [===============>..............] - ETA: 0s - loss: 0.5317 - accuracy: 0.7434
Epoch 89: saving model to checkpoints/Original\weights.89.hdf5
524/804 [==================>...........] - ETA: 0s - loss: 0.5342 - accuracy: 0.7420
Epoch 89: saving model to checkpoints/Original\weights.89.hdf5
647/804 [=======================>......] - ETA: 0s - loss: 0.5286 - accuracy: 0.7447
Epoch 89: saving model to checkpoints/Original\weights.89.hdf5
731/804 [==========================>...] - ETA: 0s - loss: 0.5302 - accuracy: 0.7436
Epoch 89: saving model to checkpoints/Original\weights.89.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7419
Epoch 90/100
 34/804 [>.............................] - ETA: 1s - loss: 0.5186 - accuracy: 0.7500
Epoch 90: saving model to checkpoints/Original\weights.90.hdf5
121/804 [=

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5578 - accuracy: 0.7252 - 626ms/epoch - 2ms/step
Loss: 0.5578128695487976, Accuracy: 0.7252478003501892


In [22]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization3/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization3/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75,callbacks=[cp_callback])

Epoch 1/75
 81/804 [==>...........................] - ETA: 1s - loss: 0.5353 - accuracy: 0.7431
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
179/804 [=====>........................] - ETA: 1s - loss: 0.5332 - accuracy: 0.7409
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
274/804 [=========>....................] - ETA: 1s - loss: 0.5324 - accuracy: 0.7412
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
381/804 [=============>................] - ETA: 0s - loss: 0.5351 - accuracy: 0.7399
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
489/804 [=================>............] - ETA: 0s - loss: 0.5340 - accuracy: 0.7419
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
590/804 [=====================>........] - ETA: 0s - loss: 0.5350 - accuracy: 0.7405
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
697/804 [=========================>....] - ETA: 0s - loss: 0.5356 - accuracy:

273/804 [=========>....................] - ETA: 1s - loss: 0.5366 - accuracy: 0.7367
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
356/804 [============>.................] - ETA: 0s - loss: 0.5378 - accuracy: 0.7377
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
463/804 [================>.............] - ETA: 0s - loss: 0.5343 - accuracy: 0.7414
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
559/804 [===================>..........] - ETA: 0s - loss: 0.5349 - accuracy: 0.7413
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
662/804 [=======================>......] - ETA: 0s - loss: 0.5352 - accuracy: 0.7414
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
764/804 [===========================>..] - ETA: 0s - loss: 0.5346 - accuracy: 0.7419
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.7415

443/804 [===============>..............] - ETA: 0s - loss: 0.5314 - accuracy: 0.7427
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
535/804 [==================>...........] - ETA: 0s - loss: 0.5310 - accuracy: 0.7433
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
641/804 [======================>.......] - ETA: 0s - loss: 0.5321 - accuracy: 0.7430
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
747/804 [==========================>...] - ETA: 0s - loss: 0.5330 - accuracy: 0.7428
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7417
Epoch 14/75
 45/804 [>.............................] - ETA: 1s - loss: 0.5362 - accuracy: 0.7479
Epoch 14: saving model to checkpoints/Optimization3\weights.14.hdf5
129/804 [===>..........................] - ETA: 1s - loss: 0.5309 - accuracy: 0.7471
Epoch 14: saving model to checkpoints/Optimizatio

612/804 [=====================>........] - ETA: 0s - loss: 0.5342 - accuracy: 0.7425
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
726/804 [==========================>...] - ETA: 0s - loss: 0.5332 - accuracy: 0.7428
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5340 - accuracy: 0.7418
Epoch 20/75
  1/804 [..............................] - ETA: 2s - loss: 0.4701 - accuracy: 0.8125
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
102/804 [==>...........................] - ETA: 1s - loss: 0.5256 - accuracy: 0.7460
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
194/804 [======>.......................] - ETA: 1s - loss: 0.5299 - accuracy: 0.7453
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
318/804 [==========>...................] - ETA: 0s - loss: 0.5322 - accuracy: 0.7434
Epoch 20: saving model to checkpoints/Optimizatio

796/804 [============================>.] - ETA: 0s - loss: 0.5332 - accuracy: 0.7416
Epoch 25: saving model to checkpoints/Optimization3\weights.25.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5336 - accuracy: 0.7412
Epoch 26/75
 88/804 [==>...........................] - ETA: 1s - loss: 0.5207 - accuracy: 0.7518
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
197/804 [======>.......................] - ETA: 1s - loss: 0.5244 - accuracy: 0.7473
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
295/804 [==========>...................] - ETA: 0s - loss: 0.5278 - accuracy: 0.7468
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
371/804 [============>.................] - ETA: 0s - loss: 0.5285 - accuracy: 0.7460
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
484/804 [=================>............] - ETA: 0s - loss: 0.5319 - accuracy: 0.7445
Epoch 26: saving model to checkpoints/Optimizatio

...

598/804 [=====================>........] - ETA: 0s - loss: 0.5329 - accuracy: 0.7422
Epoch 50: saving model to checkpoints/Optimization3\weights.50.hdf5
694/804 [========================>.....] - ETA: 0s - loss: 0.5335 - accuracy: 0.7421
Epoch 50: saving model to checkpoints/Optimization3\weights.50.hdf5
796/804 [============================>.] - ETA: 0s - loss: 0.5334 - accuracy: 0.7423
Epoch 50: saving model to checkpoints/Optimization3\weights.50.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5337 - accuracy: 0.7420
Epoch 51/75
 84/804 [==>...........................] - ETA: 1s - loss: 0.5342 - accuracy: 0.7344
Epoch 51: saving model to checkpoints/Optimization3\weights.51.hdf5
191/804 [======>.......................] - ETA: 1s - loss: 0.5359 - accuracy: 0.7363
Epoch 51: saving model to checkpoints/Optimization3\weights.51.hdf5
295/804 [==========>...................] - ETA: 0s - loss: 0.5345 - accuracy: 0.7390
Epoch 51: saving model to checkpoints/Optimizatio

767/804 [===========================>..] - ETA: 0s - loss: 0.5331 - accuracy: 0.7430
Epoch 56: saving model to checkpoints/Optimization3\weights.56.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5332 - accuracy: 0.7421
Epoch 57/75
 57/804 [=>............................] - ETA: 1s - loss: 0.5355 - accuracy: 0.7319
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
165/804 [=====>........................] - ETA: 1s - loss: 0.5421 - accuracy: 0.7318
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
268/804 [=========>....................] - ETA: 1s - loss: 0.5422 - accuracy: 0.7313
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
375/804 [============>.................] - ETA: 0s - loss: 0.5353 - accuracy: 0.7366
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
450/804 [===============>..............] - ETA: 0s - loss: 0.5359 - accuracy: 0.7384
Epoch 57: saving model to checkpoints/Optimizatio

 31/804 [>.............................] - ETA: 1s - loss: 0.5360 - accuracy: 0.7429
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
134/804 [====>.........................] - ETA: 1s - loss: 0.5257 - accuracy: 0.7463
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
235/804 [=======>......................] - ETA: 1s - loss: 0.5307 - accuracy: 0.7439
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
340/804 [===========>..................] - ETA: 0s - loss: 0.5285 - accuracy: 0.7441
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
445/804 [===============>..............] - ETA: 0s - loss: 0.5316 - accuracy: 0.7429
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
526/804 [==================>...........] - ETA: 0s - loss: 0.5316 - accuracy: 0.7423
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
634/804 [======================>.......] - ETA: 0s - loss: 0.5321 - accuracy: 0.74

222/804 [=======>......................] - ETA: 1s - loss: 0.5335 - accuracy: 0.7354
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
300/804 [==========>...................] - ETA: 0s - loss: 0.5331 - accuracy: 0.7373
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
418/804 [==============>...............] - ETA: 0s - loss: 0.5328 - accuracy: 0.7396
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
510/804 [==================>...........] - ETA: 0s - loss: 0.5298 - accuracy: 0.7436
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
618/804 [======================>.......] - ETA: 0s - loss: 0.5301 - accuracy: 0.7428
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
724/804 [==========================>...] - ETA: 0s - loss: 0.5300 - accuracy: 0.7426
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 

387/804 [=============>................] - ETA: 0s - loss: 0.5313 - accuracy: 0.7410
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
495/804 [=================>............] - ETA: 0s - loss: 0.5313 - accuracy: 0.7429
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
596/804 [=====================>........] - ETA: 0s - loss: 0.5328 - accuracy: 0.7423
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
696/804 [========================>.....] - ETA: 0s - loss: 0.5337 - accuracy: 0.7411
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
802/804 [============================>.] - ETA: 0s - loss: 0.5328 - accuracy: 0.7424
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7425


In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5745 - accuracy: 0.7258 - 438ms/epoch - 2ms/step
Loss: 0.574485719203949, Accuracy: 0.7258309125900269


In [24]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')